In [ ]:
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense

# Read TSV file and extract lines
all_data = pd.read_csv('../.cache/normalized.tsv', sep='\t', low_memory=False)
print(f"{len(all_data)} rows loaded")

all_data.head()

In [ ]:
import matplotlib.pyplot as plt

shuff = all_data #.head(1000)

count = 0

for _, row in shuff.iterrows():
    for lat, lon in [(row['lat1'], row['lon1']), (row['lat2'], row['lon2'])]:
        if lat < 0.574 or lat > .578 or lon < .798 or lon > .802:
            continue

        count += 1

        plt.scatter(lat, lon)
        if count > 300:
            break

plt.xlim()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Lines from DataFrame')
plt.show()

In [ ]:
from geo_dist_prep.model.visualize import plot_losses

features = [
    'lat1', 'lon1',
    'lat2', 'lon2',
    'angle'
]
input_shape = len(features)
predictions = [
    'by_road',
    'by_air'
]
output_shape = len(predictions)

model = Sequential([
    Dense(64, activation='relu', input_shape=(input_shape,)),
    Dense(32, activation='relu', input_shape=(input_shape,)),
    Dense(16, activation='relu', input_shape=(input_shape,)),
    Dense(output_shape, activation='linear')
])

model.compile(optimizer='adam', loss='mse')


# Split data into training and validation sets
data = all_data.copy()
# data = data.query(
#    '(0.574 <= lat1 <= 0.578) and (0.798 <= lon1 <= 0.802) and \
#    (0.574 <= lat2 <= 0.578) and (0.798 <= lon2 <= 0.802)'
# )
data["custom_sort"] = data["by_road"] / data["by_air"]
data.sort_values("custom_sort", inplace=True)

train = data.sample(frac=0.8, random_state=0)
val = data.drop(train.index)

# Split data into features and labels
train_x = train[features]
train_y = train[predictions]
val_x = val[features]
val_y = val[predictions]

# Train model
model.fit(train_x,
          train_y,
          epochs=50,
          batch_size=10_000,
          validation_data=(val_x, val_y),
          callbacks=[plot_losses]
          )

# Evaluate model
model.evaluate(val_x, val_y)

# Save model
model.save('../.cache/model.h5')


In [ ]:

southport = [53.6477, -3.0044]
st_annes = [53.7484, -3.0008]

from geo_dist_prep.geotree.node import location_to_pos

lon1, lat1 = location_to_pos(southport[0], southport[1])
lon2, lat2 = location_to_pos(st_annes[0], st_annes[1])

df = pd.DataFrame({"lat1": lat1, "lon1": lon1, "lat2": lat2, "lon2": lon2, "angle": 0}, index=[0])



prediction = model.predict()

print(prediction)